In [ ]:
import numpy as np
import pandas as pd
from datasets import concatenate_datasets, load_dataset

Reading the raw dataset.

In [ ]:
ds = load_dataset(
    "knkarthick/dialogsum",
    revision="a968e7aee0602e257935f1321a02e4287f7d5848"
)
df = concatenate_datasets(ds.values()).to_pandas()
df

Dropping useless columns.

In [ ]:
df = df.drop(columns=["id", "topic", "summary"])

Renaming columns.

In [ ]:
df = df.rename(columns={"dialogue": "body"})

Stripping the bodies.

In [ ]:
df["body"] = (df["body"].str.strip()).replace("", np.nan)

Dropping rows with missing data.

In [ ]:
df = df.dropna()

Replacing all whitespace with single spaces

In [ ]:
df["body"] = df["body"].str.replace("\\s+", " ", regex=True)

Removing the person labels.

In [ ]:
df["body"] = df["body"].str.replace("#Person\\d*#: ", "", regex=True)

Printing body duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]

Dropping the body duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")

Adding the politicalness label column.

In [ ]:
df["politicalness"] = pd.Categorical(["non-political"] * len(df))

Inspecting the body length.

In [ ]:
df["body_length"] = df["body"].fillna("").str.len()
df["body_word_count"] = df["body"].fillna("").str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

In [ ]:
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df["body_length"].iloc[::100], df["body_length"].tail(1)]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

In [ ]:
df["body_length"].mean()

In [ ]:
df.to_parquet("../preprocessed/dialogsum.parquet")